<a href="https://colab.research.google.com/github/chenpenghao/FE5112/blob/main/FE5112.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python Environment Setup

In [ ]:
#@title
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

In [ ]:
#@title
!pip install --target=$nb_path pandas

**Market:**

The market is complete and arbitrage free. The risk free interest rate is 8%. The price of a non-dividend-paying stock $S_t$ at time t is governed by the fllowing SDE under the risk neutral probability measure $\mathbb{Q}$ :
\begin{align}
d{S_t} &= 0.08 \times S_t{dt} + \sigma(S_t) \times S_t{dW_t}\\
S_0 &= 100
\end{align}

where $W_t$ is a Brownian motion under $\mathbb{Q}$ and $\sigma(\cdot)$ is a function given as follows:
\begin{equation}
  \sigma(x) = \begin{cases}
    0.25 + 0.02 \times ( 1.0 - \frac{x}{S_0}), \text{ for } x \leq 100;\\
    max(0.001, 0.25-0.01 \times (\frac{x}{S_0} - 1)), \text{ for } x > 100
  \end{cases}
\end{equation}





In [ ]:
# market specific
import math
import random
from numpy import array

T = 2     # duration in years
L = 730   # number of time intervals
M = 100   # number of asset paths
rf = 0.08 # risk free rate
S0 = 100.0  # Beginning stock price
dt = T/L  # time discretization

discf = math.exp(-rf*dt) 

def _sig(x, s0):
  if x <= 100:
    return 0.25 + 0.02 * (1.0 - x/s0)
  else:
    return max(0.001, 0.25-0.01*(x/s0 - 1))

def sig(x):
  return _sig(x, S0)

S = [[S0 for i in range(M)] for j in range(L+1)] 
for i in range(len(S)):
  for j in range(1, len(S[i])):
    dS = rf * S[i][j-1] * dt + sig(S[i][j-1])*S[i][j-1]*math.sqrt(dt)*random.gauss(0,1)
    S[i][j] = S[i][j-1] + dS

S = array(S)
print(S)






[[100.         100.11007628 100.7034864  ... 105.11688326 104.60985067
  105.01031604]
 [100.         100.99163476 100.36756995 ...  92.03470017  91.47815514
   91.00016867]
 [100.         101.07426129 103.96895888 ...  72.94490938  72.58979175
   72.2206893 ]
 ...
 [100.         100.63305723 100.8182419  ... 121.03364008 121.80246793
  120.73432199]
 [100.          98.50485193  98.32193434 ... 122.03342314 118.03534362
  118.91618196]
 [100.         100.28653446 100.80230002 ... 120.84279443 118.33629764
  119.73581341]]


**Option to Price**

It is a one-year-non-exercise American Asian put option with the strike price $K=108$ and the maturity $T=2$(years). The potion is not exercisable in the first year. The option is exercisable once and only once in the second year. If the option is exercised on day $n, 366 \leq n \leq 730$, the option buyer will receive a payout of $(K - A_n)^{+}$ where
$$A_n = \frac{1}{60}\sum^{-1}_{i=-60}S_{\frac{n+i}{365}}$$

In [ ]:
# Asset related

K = 108
def aaOption (S, dayEx):
  if dayEx < 365:
    return -1
  else:
    for i in range(len(S)):
      sPrices = S[i][dayEx-60:dayEx]
    